# Практическое задание 4
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО: Смирнов Александр Львович

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе. 

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils) 


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса. 
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой. 

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно 

## Предобработка данных (2 балла)

In [1]:
import sys
PATH_TO_CLAI_UTILS = "./utils/"
sys.path.append(PATH_TO_CLAI_UTILS)

In [2]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Setting bashlex grammar using file: /home/furiousteabag/Projects/NLP/04_generation/./utils/bashlint/grammar/grammar100.txt
Bashlint grammar set up (148 utilities)



Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [3]:
train_data = pd.read_csv('data/train.csv')
train_data

,invocation,cmd
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`
...,...,...
9938,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;
9939,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst
9940,view the manual page of find,man find
9941,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`"""


В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [4]:
test_data = pd.read_csv('data/test_data.csv')
test_data

,invocation,cmd,origin
0,create ssh connection to specified ip from spe...,ssh user123@176.0.13.154,handcrafted
1,"search for commands containing string ""zeppeli...",history | grep zeppelin,handcrafted
2,search for location of specified file or appli...,whereis python3,handcrafted
3,grant all rights to root folder,sudo chmod 777 -R /,handcrafted
4,search in running processes for specified name,ps -aux | grep zepp,handcrafted
...,...,...,...
716,"outputs the 2nd and every later field, so ""b c...","echo a b c d | cut -d"" "" -f2-",mined
717,outputs abcfgh,"echo abcdefgh | cut -c1-3,6-8",mined
718,"displays ""b"", separating fields by any number ...",echo a b|awk '{print $2}',mined
719,"displays ""ba"", preserving the order of the fie...",echo a b|awk '{print $2 $1}',mined


**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [5]:
import regex
import re

from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
stopwords = stopwords.words("english")

stemmer = PorterStemmer()

In [7]:
def clean_text(text):
    text = text.lower()
    tokens = filter(lambda x: x not in stopwords, text.split())
    tokens = map(stemmer.stem, tokens)
    text = " ".join(tokens)
    text = re.sub(r"[^a-zA-Z ]", "", text)
    return text

In [8]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [9]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

In [10]:
train_data.head()

,invocation,cmd,text_cleaned,cmd_cleaned
0,"copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...,copi loadabl kernel modul mymoduleko driver mo...,cp File $( uname -r )
1,"display all lines containing ""ip_mroute"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE,display line contain ipmroute current kernel c...,cat $( uname -r ) | grep Regex
2,display current running kernel's compile-time ...,cat /boot/config-`uname -r`,display current run kernel compiletim config file,cat $( uname -r )
3,"find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*,find loadabl modul current kernel whose name i...,find Path $( uname -r ) -regex Regex
4,"look for any instance of ""highmem"" in the curr...",grep “HIGHMEM” /boot/config-`uname -r`,look instanc highmem current kernel compiletim...,grep Regex $( uname -r )


Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [11]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [12]:
import sentencepiece as spm
import io

In [13]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2
UNK_ID = 3


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 32

In [14]:
#! mkdir -p ./tokenizers

text_tokenizer = io.BytesIO()
spm.SentencePieceTrainer.Train(
    sentence_iterator=iter(train_data["text_cleaned"]),
    #sentence_iterator=iter(train_data["invocation"]),
    model_writer=text_tokenizer,
    vocab_size=4000,
    pad_id=PAD_ID,                
    bos_id=BOS_ID,
    eos_id=EOS_ID,
    unk_id=UNK_ID
)
text_tokenizer = spm.SentencePieceProcessor(model_proto=text_tokenizer.getvalue())

cmd_tokenizer = io.BytesIO()
spm.SentencePieceTrainer.Train(
    sentence_iterator=iter(train_data["cmd_cleaned"]),
    model_writer=cmd_tokenizer,
    #model_prefix='./tokenizers/sp_cmd',
    vocab_size=500,
    pad_id=PAD_ID,                
    bos_id=BOS_ID,
    eos_id=EOS_ID,
    unk_id=UNK_ID
)
cmd_tokenizer = spm.SentencePieceProcessor(model_proto=cmd_tokenizer.getvalue())

In [15]:
text = train_data["text_cleaned"][0]
print("Initial text:")
print(text)
print("\n")

tokenized = text_tokenizer.tokenize(text, add_bos=True, add_eos=True)
print("Indexes:")
print(tokenized)
print("\n")
print("Decoded version:")
print(text_tokenizer.decode(tokenized))
print("\n")
print("Pieces:")
print(*text_tokenizer.IdToPiece(tokenized))

Initial text:
copi loadabl kernel modul mymoduleko driver modul directori matchig current kernel


Indexes:
[1, 48, 1454, 559, 431, 1253, 216, 3554, 3103, 12, 3060, 1253, 5, 30, 1826, 6, 431, 2]


Decoded version:
copi loadabl kernel modul mymoduleko driver modul directori matchig current kernel


Pieces:
<s> ▁copi ▁load abl ▁kernel ▁modul ▁my module ko ▁ driver ▁modul ▁directori ▁match ig ▁current ▁kernel </s>


In [16]:
cmd = train_data["cmd_cleaned"][0]
print("Initial cmd:")
print(cmd)
print("\n")

tokenized = cmd_tokenizer.tokenize(cmd, add_bos=True, add_eos=True)
print("Indexes:")
print(tokenized)
print("\n")
print("Decoded version:")
print(cmd_tokenizer.decode(tokenized))
print("\n")
print("Pieces:")
print(*cmd_tokenizer.IdToPiece(tokenized))

Initial cmd:
cp File $( uname -r )


Indexes:
[1, 84, 12, 14, 45, 26, 210, 11, 4, 41, 31, 2]


Decoded version:
cp File $( uname -r )


Pieces:
<s> ▁cp ▁File ▁ $ ( ▁u name ▁- r ▁) </s>


**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [17]:
class TextToBashDataset(Dataset):
    
    def __init__(self, texts, cmds, text_tokenizer, cmd_tokenizer,
                 max_text_length=MAX_TEXT_LENGTH, max_code_length=MAX_CODE_LENGTH):
        
        self.text_tokenizer = text_tokenizer
        self.cmd_tokenizer = cmd_tokenizer
        self.max_text_length = max_text_length
        self.max_code_length = max_code_length
        
        self.items = []
        
        for text, cmd in zip(texts, cmds):
            text_tokenized = text_tokenizer.tokenize(text, add_bos=True, add_eos=True)
            cmd_tokenized = cmd_tokenizer.tokenize(cmd, add_bos=True, add_eos=True)
            if len(text_tokenized) > max_text_length:
                text_tokenized = text_tokenized[:max_text_length-1] + text_tokenized[-1:]
            if len(cmd_tokenized) > max_code_length:
                cmd_tokenized = cmd_tokenized[:max_code_length-1] + cmd_tokenized[-1:]
            self.items.append((text_tokenized, cmd_tokenized))
            
    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

In [18]:
train_ds = TextToBashDataset(
    texts=train_data["text_cleaned"],
    cmds=train_data["cmd_cleaned"],
    text_tokenizer=text_tokenizer,
    cmd_tokenizer=cmd_tokenizer)

valid_ds = TextToBashDataset(
    texts=valid_data["text_cleaned"],
    cmds=valid_data["cmd_cleaned"],
    text_tokenizer=text_tokenizer,
    cmd_tokenizer=cmd_tokenizer)

In [19]:
text_idx, cmd_idx = valid_ds[0]
print(valid_data.iloc[0])
print("\n")
print(text_idx)
print(text_tokenizer.decode(text_idx))
print(cmd_tokenizer.decode(cmd_idx))

invocation      searches through the root filesystem ("/") for...
cmd                          find / -name Chapter1 -type f -print
text_cleaned    search root filesystem  file name chapter prin...
cmd_cleaned                  find Path -name Regex -type f -print
Name: 9843, dtype: object


[1, 13, 76, 179, 4, 9, 890, 11, 162, 2]
search root filesystem file name chapter print locat
find Path -name Regex -type f -print


In [20]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):

    text_idxs = [torch.tensor(item[0]) for item in batch]
    cmd_idxs = [torch.tensor(item[1]) for item in batch]
    
    text_idxs = pad_sequence(text_idxs, padding_value=PAD_ID, batch_first=True)
    cmd_idxs = pad_sequence(cmd_idxs, padding_value=PAD_ID, batch_first=True)
    
    return text_idxs, cmd_idxs 

In [21]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn),
}

In [22]:
next(iter(loaders["valid"]))

(tensor([[   1,   13,   76,  179,    4,    9,  890,   11,  162,    2,    0,    0,
             0,    0,    0,    0,    0],
         [   1,   13,   76,  179,    4,    9,  890,    2,    0,    0,    0,    0,
             0,    0,    0,    0,    0],
         [   1,   13,   76,  179,    4,    9,  890,    2,    0,    0,    0,    0,
             0,    0,    0,    0,    0],
         [   1,   13,    4,   80,  183,    2,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0],
         [   1,   45, 1633,   42,  171,  158, 1125,    2,    0,    0,    0,    0,
             0,    0,    0,    0,    0],
         [   1,   45,  789,   38,  350,  225, 3274, 2922,    6,    5,  109,   42,
          3079,    2,    0,    0,    0],
         [   1,   45,   29,  750,  223,  211,  823, 1200,   74,  481,  166,    2,
             0,    0,    0,    0,    0],
         [   1,   45,   29,  310,    6,  145,  431, 2208,  481,  736,    2,    0,
             0,    0,    0,    0,    0],
         [   1, 

## Обучение бейзлайна (2 балла)

In [23]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [24]:
text_model_config = BertConfig(
    vocab_size = text_tokenizer.vocab_size(),
    hidden_size = 256,
    num_hidden_layers = 2,
    num_attention_heads = 8,
    intermediate_size = 256 * 4,
    hidden_dropout_prob = 0.1,
    pad_token_id = PAD_ID,
)

cmd_model_config = BertConfig(
    vocab_size = cmd_tokenizer.vocab_size(),
    hidden_size = 256,
    num_hidden_layers = 2,
    num_attention_heads = 8,
    intermediate_size = 256 * 4,
    hidden_dropout_prob = 0.1,
    pad_token_id = PAD_ID,
    is_decoder = True,
    add_cross_attention = True
)

print(cmd_model_config.is_decoder)
print(cmd_model_config.add_cross_attention)

True
True


In [25]:
config = EncoderDecoderConfig.from_encoder_decoder_configs(text_model_config, cmd_model_config)
model = EncoderDecoderModel(config=config)

In [26]:
text_idxs, cmd_idxs = next(iter(loaders["valid"]))
decoder_input = cmd_idxs[..., :-1]
target = cmd_idxs[..., 1:]

print(text_idxs.shape)
print(decoder_input.shape)
print(target.shape)

torch.Size([32, 17])
torch.Size([32, 38])
torch.Size([32, 38])


In [27]:
text_idxs_mask = torch.where(text_idxs != PAD_ID, 1, 0)
decoder_input_mask = torch.where(decoder_input != PAD_ID, 1, 0)

In [28]:
cmd_idxs[..., :-1]

tensor([[1, 6, 8,  ..., 0, 0, 0],
        [1, 6, 8,  ..., 0, 0, 0],
        [1, 6, 8,  ..., 0, 0, 0],
        ...,
        [1, 6, 8,  ..., 0, 0, 0],
        [1, 6, 8,  ..., 0, 0, 0],
        [1, 6, 8,  ..., 0, 0, 0]])

In [29]:
out = model(
    input_ids=text_idxs, decoder_input_ids=decoder_input,
    attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)

logits = out.logits
print(logits.shape)

torch.Size([32, 38, 500])


### eval

In [30]:
DEVICE = torch.device("cuda")

In [31]:
def greedy_decode(text_tokenized, model, max_len=20):
    
    model.eval()
    model = model.to(DEVICE)
    text_tokenized = text_tokenized.unsqueeze(0).to(DEVICE)
    cmd_prediction = torch.tensor([BOS_ID]).unsqueeze(0).to(DEVICE)
    next_piece = BOS_ID
    
    while next_piece != EOS_ID and cmd_prediction.shape[-1] < max_len:
        #print(cmd_prediction)
        out = model(input_ids=text_tokenized, decoder_input_ids=cmd_prediction)
        next_piece = torch.argmax(out.logits.squeeze(0)[-1]).item()
        cmd_prediction = torch.cat((cmd_prediction, torch.tensor([[next_piece]]).to(DEVICE)), dim=1)
        
    return cmd_prediction[0]

In [32]:
def eval_decode(i, data, ds, model, max_len=20):
    
    for k, v in dict(data.iloc[i]).items():
        print(f"{k}: {v}")
    text_tokenized = torch.tensor(ds[i][0])
    cmd_prediction = greedy_decode(text_tokenized, model, max_len)
    cmd_prediction = [x.item() for x in cmd_prediction]
    print(f"cmd_predicted: {cmd_tokenizer.decode(cmd_prediction)}")

In [33]:
import random

def random_eval(data, ds, model, sample_size=5, max_len=20):
    indexes = random.sample(range(len(ds)), sample_size)
    for i in indexes:
        eval_decode(i, data, ds, model, max_len)
        print()

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [34]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=3, min_delta=0.01):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [35]:
import os
from datetime import datetime
import shutil
from collections import defaultdict

import torch
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm, trange

#from apex import amp

class Trainer:

    def __init__(
            self, 
            model, 
            criterion,
            optimizer, 
            pad_token_id,
            device,
            stopper,
            logdir=None,
            max_grad_norm=-1
    ):
        """
            model: объект класса BertModel
            optimizer: оптимизатор
            pad_token_id: индекс паддинга. Нужен для создания attention mask
            device: девайс (cpu или cuda), на котором надо производить вычисления
            logdir: директория для записи логов
            max_grad_norm: максимум нормы градиентов, для клиппинга
        """
        self._criterion = criterion
        self._optimizer = optimizer
        self._pad_token_id = pad_token_id
        self._device = device
        self._logdir = logdir
        self._stopper = stopper
        self._max_grad_norm = max_grad_norm
        
        self._model = model.to(self._device)
        
        if self._logdir is not None:
            self._writer = SummaryWriter(log_dir=f"{logdir}/{datetime.now()}/", flush_secs=1)

        self._n_epoch = 0
        self._n_iter = 0

    def train(self, dataloaders, n_epochs):
        for epoch in trange(n_epochs):
            train_loss = self._train_step(dataloaders["train"])
            val_loss = self._val_step(dataloaders["valid"])
            self._n_epoch += 1
            print(f"Epoch: {self._n_epoch} | train_loss: {train_loss:.3f} | val_loss: {val_loss:.3f}")
            
            if self._logdir is not None:
                self._writer.add_scalar("loss/train", train_loss, global_step=self._n_epoch)
                self._writer.add_scalar("loss/val", val_loss, global_step=self._n_epoch)
                
            self._stopper(val_loss)
            torch.save(self._model.state_dict(), f"./checkpoints/{val_loss}.pt")
            
            trainer._stopper.best_loss
            if self._stopper.early_stop:
                break
        
        self._model.load_state_dict(torch.load(f"./checkpoints/{self._stopper.best_loss}.pt"))

    def _train_step(self, dataloader):
        """
            dataloader: объект класса DataLoader для обучения
        """
        self._model.train()
        epoch_loss = 0
        for text_idxs, cmd_idxs in dataloader:
            
            text_idxs = text_idxs.to(self._device)
            decoder_input = cmd_idxs[..., :-1].to(self._device)
            target = cmd_idxs[..., 1:].to(self._device)
            
            text_idxs_mask = torch.where(text_idxs != self._pad_token_id, 1, 0).to(self._device)
            decoder_input_mask = torch.where(decoder_input != self._pad_token_id, 1, 0).to(self._device)
            
            self._optimizer.zero_grad()        
            
            out = self._model(
                input_ids=text_idxs, decoder_input_ids=decoder_input,
                attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)

            logits = out.logits

            loss = self._criterion(logits.reshape(-1, cmd_tokenizer.vocab_size()), target.reshape(-1))
            epoch_loss += loss.item()
            
            loss.backward()
            
            if self._max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(self._model.parameters(), self._max_grad_norm)
                
            self._optimizer.step()
        return epoch_loss / len(dataloader)
    
    def _val_step(self, dataloader):
        """
            dataloader: объект класса DataLoader для обучения
        """
        self._model.eval()
        epoch_loss = 0
        for text_idxs, cmd_idxs in dataloader:
            
            text_idxs = text_idxs.to(self._device)
            decoder_input = cmd_idxs[..., :-1].to(self._device)
            target = cmd_idxs[..., 1:].to(self._device)
            
            text_idxs_mask = torch.where(text_idxs != self._pad_token_id, 1, 0).to(self._device)
            decoder_input_mask = torch.where(decoder_input != self._pad_token_id, 1, 0).to(self._device)
            
            with torch.no_grad():
                out = self._model(
                    input_ids=text_idxs, decoder_input_ids=decoder_input,
                    attention_mask=text_idxs_mask, decoder_attention_mask=decoder_input_mask)
                logits = out.logits
                loss = self._criterion(logits.reshape(-1, cmd_tokenizer.vocab_size()), target.reshape(-1))
            
            epoch_loss += loss.item()
            
        return epoch_loss / len(dataloader)

In [36]:
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
stopper = EarlyStopping(patience=5, min_delta=0.01)

In [37]:
device = torch.device('cuda')

In [38]:
trainer = Trainer(
    model=model, 
    criterion=criterion,
    optimizer=optimizer, 
    pad_token_id=PAD_ID,
    device=device,
    logdir="./runs",
    stopper=stopper,
    max_grad_norm=-1
)

In [39]:
! mkdir -p ./checkpoints

Переучивать не будем:

![](./training.png)

Видим, что лучший лосс был на 8-й эпохе.
Подгрузим эти веса (при переобучении модель автоматически в конце подгружает лучшие веса):

In [40]:
model.load_state_dict(torch.load("./checkpoints/0.9302761256694794.pt"))
trainer._model = model
print(f"val_loss: {round(trainer._val_step(loaders['valid']), 3)}")

val_loss: 0.93


## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей. 

In [41]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1.5,
            device=torch.device('cuda'),
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        
        self.device = device
        
    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        
        chains = torch.full([self.beam_width, 1], self.bos_id).to(self.device)
        chain_probabilities = torch.zeros(self.beam_width).to(self.device)
        if_chain_ready = torch.full([self.beam_width], False).to(self.device)
        
        # saving encoder outputs
        input_text_tokens = input_text_tokens.repeat([self.beam_width, 1]).to(device)
        encoder_outputs = model._prepare_encoder_decoder_kwargs_for_generation(
            input_ids=input_text_tokens,
            model_kwargs={})["encoder_outputs"]
        
        idx = 0
        while idx <= self.max_length and not all(if_chain_ready):
            
            logits = model(encoder_outputs=encoder_outputs, decoder_input_ids=chains).logits[:, -1, :]
            
            probs = torch.log_softmax(logits / self.temperature, dim=-1)
            probs, tokens = torch.topk(probs, k=self.beam_width)
            
            if idx == 0:
                # pick all top tokens
                # 0 because current chains are same
                beam_tokens = tokens[0].view(self.beam_width, 1)
                chains = torch.cat([chains, beam_tokens], dim=-1)
                chain_probabilities += probs[0]
                idx += 1
                continue

            chain_probabilities_beam = chain_probabilities.reshape(-1, 1).repeat([1, self.beam_width])
            ready_chains_idxs = if_chain_ready.nonzero()
            
            # if chain is already ready,
            # keep one instance of it and add
            # padding
            probs[ready_chains_idxs, :] = -float("inf")
            probs[ready_chains_idxs, 0] = 0
            tokens[ready_chains_idxs, :] = self.pad_id

            chain_probabilities_beam = chain_probabilities_beam + probs
            
            # choosing best across all options
            best_sequences = torch.argsort(chain_probabilities_beam.flatten())[-self.beam_width:]

            chains = torch.cat([chains[best_sequences // self.beam_width, :], tokens.flatten()[best_sequences].view(-1, 1)], dim=-1)
            chain_probabilities = chain_probabilities_beam.flatten()[best_sequences]

            if_chain_ready = ((chains[:, -1] == self.eos_id) | (chains[:, -1] == self.pad_id))
                
            idx += 1
        
        return list(zip(chains, chain_probabilities))        

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [42]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=1, device=DEVICE
)

In [43]:
all_scores = []
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('text cleaned:', valid_data.text_cleaned.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = torch.tensor(valid_ds[i][0])
        pred = beam_search_engine.get_result(model, src)
        
        #print('greedy decode:', cmd_tokenizer.decode(list(map(int, greedy_decode(src, model)))))
        scores = []
        for x, proba in pred[:5]:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))
        all_scores.append(max(scores))
        
print(np.mean(all_scores))


text: searches through the root filesystem ("/") for the file named chapter1, and prints the location
text cleaned: search root filesystem  file name chapter print locat
true: find / -name Chapter1 -type f -print
true cleaned: find Path -name Regex -type f -print
find Path -name Regex -type f -printf "%f\n" tensor(-4.4631, device='cuda:0')
find Path -name Regex -or -name Regex tensor(-3.2608, device='cuda:0')
find Path -name Regex -type f -print tensor(-2.9428, device='cuda:0')
find Path -name Regex -type f tensor(-1.5682, device='cuda:0')
find Path -name Regex tensor(-0.7397, device='cuda:0')
1.0

text: searches through the root filesystem ("/") for the file named chapter1.
text cleaned: search root filesystem  file name chapter
true: find / -name Chapter1 -type f
true cleaned: find Path -name Regex -type f
find Path -name Regex -or -name Regex -type f tensor(-4.5106, device='cuda:0')
find Path -name Regex -or -name Regex tensor(-2.8590, device='cuda:0')
find Path -name Regex -type f

/home/furiousteabag/.local/share/virtualenvs/base/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


find Path -name Regex -or -name Regex -type f tensor(-4.5106, device='cuda:0')
find Path -name Regex -or -name Regex tensor(-2.8590, device='cuda:0')
find Path -name Regex -type f -print tensor(-2.0952, device='cuda:0')
find Path -name Regex -type f tensor(-1.5648, device='cuda:0')
find Path -name Regex tensor(-0.8799, device='cuda:0')
1.0

text: searching for all files with the extension mp3
text cleaned: search file extens mp
true: find / -name *.mp3
true cleaned: find Path -name Regex
find Path -type f -iname Regex -print tensor(-4.9602, device='cuda:0')
find Path -type f -iname Regex tensor(-2.0453, device='cuda:0')
find Path -iname Regex tensor(-1.9404, device='cuda:0')
find Path -name Regex tensor(-1.7023, device='cuda:0')
find Path -type f -name Regex tensor(-1.6675, device='cuda:0')
1.0

text: set myvariable to the value of variable_name
text cleaned: set myvari valu variablenam
true: myVariable=$(env  | grep VARIABLE_NAME | grep -oe '[^=]*$');
true cleaned: env | grep Regex | 

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [44]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for i, (text, target_cmd) in tqdm(enumerate(zip(df.text_cleaned.values, df.cmd.values)), total=len(df)):
        
        input_tokens = text_tokenizer.tokenize(text, add_bos=True, add_eos=True)
        if len(input_tokens) > MAX_TEXT_LENGTH:
            input_tokens = input_tokens[:MAX_TEXT_LENGTH-1] + input_tokens[-1:]
        input_tokens = torch.tensor(input_tokens)
        
        predictions = beam_engine.get_result(model, input_tokens)
        
        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)
        
        all_scores.append(max(object_scores))
    return all_scores

Если вы всё реализовали правильно, подобрали параметры BeamSearch то ваш средний скор на валидации должен быть >= 0.25, а скор на `handcrafted` части теста >= 0.13. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

In [45]:
def find_best_temperature(model, valid_data):
    scores = []
    temperatures = np.arange(0.5, 2.1, 0.1)
    for temperature in temperatures:
        beam_search_engine = BeamSearchGenerator(
            pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
            max_length=MAX_CODE_LENGTH, beam_width=5,
            temperature=temperature, device=DEVICE
        )
        scores.append(np.mean(compute_all_scores(model, valid_data, beam_search_engine)))
    return temperatures[np.argmax(scores)]

![](./searching_temperature.png)

In [46]:
beam_search_engine = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=1.5, device=DEVICE
)

In [47]:
val_scores = compute_all_scores(model, valid_data, beam_search_engine)
print(np.mean(val_scores))

100%|██████████| 100/100 [00:09<00:00, 10.68it/s]

0.26862499999999995


In [48]:
test_scores = compute_all_scores(model, test_data[test_data["origin"] == "handcrafted"], beam_search_engine)
print(np.mean(test_scores))

100%|██████████| 129/129 [00:11<00:00, 10.93it/s]

0.1491279069767442


In [49]:
mined_scores = compute_all_scores(model, test_data[test_data["origin"] == "mined"], beam_search_engine)
print(np.mean(mined_scores))

100%|██████████| 592/592 [00:49<00:00, 11.89it/s]

-0.27850573788073785


## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на `mined` >= 0 при скоре на `handrafted` >= 0.16.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...